In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import os, glob
import os.path

In [2]:
kliniklist = pd.read_excel(r'..\Klinikliste.xlsx')
df_kliniklist = pd.DataFrame(kliniklist, columns= ['Klinikname','Link Google Maps','Link Klinikbewertungen'])
df_kliniklist['klinik_ID'] = [i for i in range(1,25)]
df_kliniklist.reset_index(drop=True, inplace=True)

In [3]:
df_kliniklist.head(30)

,Klinikname,Link Google Maps,Link Klinikbewertungen,klinik_ID
0,Augenklinik Dr.Hoffmann,https://www.google.com/maps/place/Augenklinik+...,https://www.klinikbewertungen.de/klinik-forum/...,1
1,Krankenhaus Marienstift Braunschweig,https://www.google.com/maps/place/Krankenhaus+...,https://www.klinikbewertungen.de/klinik-forum/...,2
2,Herzogin Elisabeth Hospital Braunschweig,https://www.google.com/maps/place/Herzogin+Eli...,https://www.klinikbewertungen.de/klinik-forum/...,3
3,Asklepios Fachklinikum Göttingen,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...,4
4,Asklepios Fachklinikum Tiefenbrunn,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...,5
5,Friederikenstift Hannover,https://www.google.com/maps/place/DIAKOVERE+Fr...,https://www.klinikbewertungen.de/klinik-forum/...,6
6,Annastift Hannover,https://www.google.com/maps/place/DIAKOVERE+An...,https://www.klinikbewertungen.de/klinik-forum/...,7
7,Clementinenhaus,https://www.google.com/maps/place/DRK-Krankenh...,https://www.klinikbewertungen.de/klinik-forum/...,8
8,Sophienklinik Hannover,https://www.google.com/maps/place/Sophienklini...,https://www.klinikbewertungen.de/klinik-forum/...,9
9,KRH Klinikum Großburgwedel,https://www.google.com/maps/place/KRH+Klinikum...,https://www.klinikbewertungen.de/klinik-forum/...,10


In [4]:
def exchange_column(old_df, new_df, column):
    old_df.drop(labels=column, axis="columns", inplace=True)
    old_df[column] = new_df[column]
    new_df = pd.DataFrame()

#### Cookies Akzeptieren

In [5]:
def accept_cookies():
    try:
        cookies=driver.find_element_by_id("ez-accept-all")
        cookies.click()
    except:
        driver.quit()

#### Scrolling Down and Open "Weitere Bewertungen"

In [6]:
def take_all_bewertung():
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        mehr=driver.find_element_by_link_text("Weitere Bewertungen anzeigen...")
        mehr.click()
    except:
        pass

#### 'Bewertungen Anzahl' mit "Datum" as Reference

In [7]:
def anzahl():
    bewertung_datum = driver.find_elements_by_css_selector("div.meta > time")
    return int(len(bewertung_datum))
    #x=1
    #for i in bewertung_datum:
    #    x=x+1
    #return x

#### Match Klinik_ID with Bewertung_ID

In [8]:
def choose_klinik(ref_value):
    return df_kliniklist.loc[df_kliniklist['Link Klinikbewertungen']==ref_value]['Klinikname'].to_string().split(" ", 1)[1].lstrip()

In [9]:
def choose_klinik_id(ref_value):
    return df_kliniklist.loc[df_kliniklist['Link Klinikbewertungen']==ref_value]['klinik_ID'].to_string().split(" ", 1)[1].lstrip()

#### Fill in the Klinik liste

In [10]:
#Klinik Name
# klinik_name=driver.find_element_by_css_selector('.klinik-normal > header > h1').text 

def name_klinik_klinik_ID(klinik_name, klinik_id):
    name_klinik = []
    klin_id = []
    for i in range(bew_anzahl):
        name_klinik.append(klinik_name)
        klin_id.append(klinik_id)

    temp_df['Name_Klinik'] =  name_klinik
    exchange_column(df,temp_df,'Name_Klinik')      

    temp_df['Klinik_ID'] =  klin_id
    exchange_column(df,temp_df,'Klinik_ID')

#### Assign Bewertung_ID

In [11]:
def bewertung_ID_erstell():
    df["Bewertung_ID"] = [i for i in range(1,bew_anzahl+1)]

#### Saving All Comments in a List

In [12]:
def saving_all_comments_in_a_list():

    titels=driver.find_element_by_class_name("ratinglist").find_elements_by_tag_name("h2")

    fach_bereich=driver.find_elements_by_css_selector("div.ratinglist > article.bewertung > span")

    bewertung_datum=driver.find_elements_by_css_selector("div.meta > time")

    gesamte_sterne=driver.find_elements_by_css_selector(".rating > dl > dd:nth-child(2) > img")
    #for i in gesamte_sterne:
    # print(int((i.get_attribute("class")).strip("star- ")))

    qualitat_sterne=driver.find_elements_by_css_selector(".bewertung > section.rating > dl > dd:nth-child(6) > img")

    behandlung_sterne=driver.find_elements_by_css_selector(".bewertung > section.rating > dl > dd:nth-child(6) > img")

    verwaltung_sterne=driver.find_elements_by_css_selector(".bewertung > section.rating > dl > dd:nth-child(8) > img")

    gestaltung_sterne=driver.find_elements_by_css_selector(".bewertung > section.rating > dl > dd:nth-child(10) > img")

    bewertungen=driver.find_elements_by_css_selector('.bewertung > section.report > dl:nth-child(2) > dd > p')
    
    return titels, fach_bereich, bewertung_datum, gesamte_sterne, qualitat_sterne, behandlung_sterne, verwaltung_sterne, gestaltung_sterne, bewertungen

#### Filna und NONE und Style und CSV

In [13]:
def fillna_and_none(klinik_id,df):
    df= df.replace(r'', np.NaN)
    df = df.fillna(value=np.nan)
    df = df[['Bewertung_ID', 'Klinik_ID','Name_Klinik','Fach_Bereich','Titel','Bewertung_Datum','Textuelle_Bewertung','Gesamte_Sterne','Qualitaet','Behandlung','Verwaltung','Gestaltung']]
    path = r'csv\\' + klinik_id + '.csv'
    df.to_csv(path, encoding="utf-8", index=False)
    driver.close()

#### fach_bereich

In [14]:
def collect_all_fach_bereich(fach_bereich):
    fach_field = []
    for i in fach_bereich:
        fach_field.append(i.text)
    temp_df['Fach_Bereich'] =  fach_field
    exchange_column(df,temp_df,'Fach_Bereich')

#### Collect All bewertung_datum

In [15]:
def collect_all_bewertung_datum(bewertung_datum):
    all_bew_date = []
    for i in bewertung_datum:
        all_bew_date.append(i.text)
    temp_df['Bewertung_Datum'] =  all_bew_date
    exchange_column(df,temp_df,'Bewertung_Datum')

#### Collect All Bewertungen

In [16]:
def collect_all_bewertungen(bewertungen):
    all_comments = []
    for i in bewertungen:
        all_comments.append(i.text)
    temp_df['Textuelle_Bewertung'] =  all_comments
    exchange_column(df,temp_df,'Textuelle_Bewertung')

#### Gestaltung

In [17]:
def collect_all_gestaltung_sterne(gestaltung_sterne):
    ges_sterne = []
    for i in gestaltung_sterne:
        ges_sterne.append(int((i.get_attribute("class")).strip("star- ")))
        
    temp_df['Gestaltung'] =  ges_sterne
    exchange_column(df,temp_df,'Gestaltung')

#### verwaltung_sterne

In [18]:
def collect_all_verwaltung_sterne(verwaltung_sterne):
    ver_sterne = []
    for i in verwaltung_sterne:
        ver_sterne.append(int((i.get_attribute("class")).strip("star- ")))
    temp_df['Verwaltung'] =  ver_sterne
    exchange_column(df,temp_df,'Verwaltung')

#### behandlung_sterne

In [19]:
def collect_all_beh_sterne(beh_sterne):
    bh_sterne = []
    for i in beh_sterne:
        bh_sterne.append(int((i.get_attribute("class")).strip("star- ")))
    temp_df['Behandlung'] =  bh_sterne
    exchange_column(df,temp_df,'Behandlung')

#### Collect All Titels

In [20]:
def collect_all_likes(title):
    titels = []
    for i in title:
        titels.append(i.text)
    temp_df['Titel'] =  titels
    exchange_column(df,temp_df,'Titel')

#### Gesamte Sterne

In [21]:
def collect_all_gesamte_sterne(gesamte_sterne):
    gesa_sterne = []
    for i in gesamte_sterne:
        gesa_sterne.append(int((i.get_attribute("class")).strip("star- ")))
    temp_df['Gesamte_Sterne'] =  gesa_sterne
    exchange_column(df,temp_df,'Gesamte_Sterne')    

#### Qualitat_sterne

In [22]:
def collect_all_qual_sterne(qual_sterne):
    qu_sterne = []
    for i in qual_sterne:
        qu_sterne.append(int((i.get_attribute("class")).strip("star- ")))
    temp_df['Qualitaet'] =  qu_sterne
    exchange_column(df,temp_df,'Qualitaet')

In [23]:
for i in df_kliniklist['Link Klinikbewertungen']:
    try:
        list_of_elements = []
        column_names=['Klinik_ID','Name_Klinik','Fach_Bereich','Titel','Bewertung_Datum','Textuelle_Bewertung','Gesamte_Sterne','Qualitaet','Behandlung','Verwaltung','Gestaltung']
        df=pd.DataFrame(columns=column_names)
        temp_df = pd.DataFrame()
        url=i
        driver=webdriver.Chrome(executable_path="../chromedriver")
        driver.get(url)
        accept_cookies()
        take_all_bewertung()
        bew_anzahl=anzahl()
        list_of_elements = saving_all_comments_in_a_list()
        collect_all_likes(list_of_elements[0])
        collect_all_fach_bereich(list_of_elements[1])
        collect_all_bewertung_datum(list_of_elements[2])
        collect_all_gesamte_sterne(list_of_elements[3][1:])
        collect_all_qual_sterne(list_of_elements[4])
        collect_all_beh_sterne(list_of_elements[5])
        collect_all_verwaltung_sterne(list_of_elements[6])
        #collect_all_gestaltung_sterne(list_of_elements[7])
        collect_all_bewertungen(list_of_elements[8])

        klinik_name = choose_klinik(i)
        klnk_id = choose_klinik_id(i)
        name_klinik_klinik_ID(klinik_name, klnk_id)
        bewertung_ID_erstell()
        fillna_and_none(klinik_name,df)
        print('Success')
    except:
        driver.close()
        print('End Mit Fehler(Übersprungen)')

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


#### Merge All Files in One

In [24]:
path = "csv/"
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "csv/merged_all.csv", encoding="utf-8", index=False)

In [26]:
df_all = pd.read_csv('csv/merged_all.csv')

In [27]:
df_all

,Bewertung_ID,Klinik_ID,Name_Klinik,Fach_Bereich,Titel,Bewertung_Datum,Textuelle_Bewertung,Gesamte_Sterne,Qualitaet,Behandlung,Verwaltung,Gestaltung
0,1,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,Schmerztherapie,Katastrophale Zustände bei der Besucherregelung!!,20.03.2021,Ich selbst bin Patientenfürsprecherin einer Kl...,0,4,4,0,NaN
1,2,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,Innere,Hoffentlich nie wieder,03.03.2021,Sonntag Abend in der Notaufnahme mein 14jährig...,0,0,0,0,NaN
2,3,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,Handchirurgie,Hand OP,18.02.2021,Ich möchte mich ganz herzlich bedanken bei den...,6,6,6,6,NaN
3,4,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,Frauen,Sehr gute Behandlung,10.02.2021,Also ich kann die Station N 21 defenitiv nur l...,6,6,6,6,NaN
4,5,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,Entbindung,Unbeschreibliche Schwestern,31.01.2021,Hallo ich bedanke mich sehr sehr für diesen Em...,6,6,6,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2999,73,9,Sophienklinik Hannover,Chirurgie,dilettantenstadl,19.08.2007,das 60´er jahre- flair wirkt nicht einladend. ...,0,2,2,0,NaN
3000,74,9,Sophienklinik Hannover,Innere,"Als Kassenpatientin auch hilfreich, aber irgen...",11.08.2007,"Als Kassenpatientin in einem 5 Bett Zimmer, wa...",2,2,2,2,NaN
3001,75,9,Sophienklinik Hannover,Innere,Für Privatpatienten sicher gut geeignet! Kasse...,11.08.2007,"Ich war da mal als Privatpatientin, es ist abe...",4,4,4,4,NaN
3002,76,9,Sophienklinik Hannover,Handchirurgie,Die schlechteste Klinik in Hannover überhaupt!!!,08.02.2007,NaN,0,0,0,0,NaN
